In this kernel, I am working on Lyft sample dataset. I have tried merging frame dataset with scene dataset, so that I could replicate the scenes both with and without using Rastiser library provided by L5Kit. Also using the curated dataset, I will try to do some feature engineering and train models on it. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os

'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
 

In [ ]:
import l5kit 

from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset

from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points

from l5kit.data import PERCEPTION_LABELS

import zarr
import os

l5kit.__version__

In [ ]:
os.environ["L5KIT_DATA_FOLDER"] = "/kaggle/input/lyft-motion-prediction-autonomous-vehicles"
# get config
cfg = load_config_data("/kaggle/input/lyft-config-files/visualisation_config.yaml")
cfg1 = load_config_data("/kaggle/input/lyft-config-files/agent_motion_config.yaml")
print(cfg1)

In [ ]:
data = zarr.open("/kaggle/input/lyft-motion-prediction-autonomous-vehicles/scenes/train.zarr/")
print(data.info)

In [ ]:
dm = LocalDataManager()
#dataset_path = dm.require(cfg["val_data_loader"]["key"])
dataset_path = '/kaggle/input/lyft-motion-prediction-autonomous-vehicles/scenes/sample.zarr'
zarr_dataset = ChunkedDataset(dataset_path)
zarr_dataset.open()
print(zarr_dataset)

In [ ]:
train_cfg = cfg1["train_data_loader"]
rasterizer = build_rasterizer(cfg, dm)
agent_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
agent_dataset = AgentDataset(cfg, agent_zarr, rasterizer)
print(agent_dataset)

Reading agents, frames, scenes and traffic light dataset into csv files

In [ ]:
agents = pd.DataFrame.from_records(zarr_dataset.agents, columns = ['centroid', 'extent', 'yaw', 'velocity', 'track_id', 'label_probabilities'])
frames = pd.DataFrame.from_records(zarr_dataset.frames, columns = ['timestamp','agent_index_interval','traffic_light_faces_index_interval','ego_translation','ego_rotation'])
scenes = pd.DataFrame.from_records(zarr_dataset.scenes, columns = ['frame_index_interval', 'host', 'start_time','end_time'])
traffic_light = pd.DataFrame.from_records(zarr_dataset.tl_faces, columns = ['face_id','traffic_light_id','traffic_light_face_status'])

In [ ]:
agents[['centroid_x','centroid_y']] = agents.centroid.tolist()
agents[['extent_x','extent_y','extent_z']] = agents.extent.tolist()
agents[['velocity_x', 'velocity_y']] =  agents.velocity.tolist()
agents.drop(['centroid','extent','velocity'],axis = 'columns',inplace = True)
PERCEPTION_LABELS = [
    "PERCEPTION_LABEL_NOT_SET",
    "PERCEPTION_LABEL_UNKNOWN",
    "PERCEPTION_LABEL_DONTCARE",
    "PERCEPTION_LABEL_CAR",
    "PERCEPTION_LABEL_VAN",
    "PERCEPTION_LABEL_TRAM",
    "PERCEPTION_LABEL_BUS",
    "PERCEPTION_LABEL_TRUCK",
    "PERCEPTION_LABEL_EMERGENCY_VEHICLE",
    "PERCEPTION_LABEL_OTHER_VEHICLE",
    "PERCEPTION_LABEL_BICYCLE",
    "PERCEPTION_LABEL_MOTORCYCLE",
    "PERCEPTION_LABEL_CYCLIST",
    "PERCEPTION_LABEL_MOTORCYCLIST",
    "PERCEPTION_LABEL_PEDESTRIAN",
    "PERCEPTION_LABEL_ANIMAL",
    "AVRESEARCH_LABEL_DONTCARE"]   #only these perception labels were true hence using only these values
agents[PERCEPTION_LABELS]= agents['label_probabilities'].tolist()

In [ ]:
agents.drop(columns = ['PERCEPTION_LABEL_NOT_SET', 'AVRESEARCH_LABEL_DONTCARE', 'PERCEPTION_LABEL_ANIMAL', 'PERCEPTION_LABEL_MOTORCYCLE', 'PERCEPTION_LABEL_MOTORCYCLIST', 'PERCEPTION_LABEL_BICYCLE',  'PERCEPTION_LABEL_OTHER_VEHICLE', 'PERCEPTION_LABEL_EMERGENCY_VEHICLE', 'PERCEPTION_LABEL_DONTCARE', 'PERCEPTION_LABEL_VAN', 'PERCEPTION_LABEL_TRAM', 'PERCEPTION_LABEL_BUS', 'PERCEPTION_LABEL_TRUCK'], inplace = True)
agents.drop(columns = ['label_probabilities'], inplace = True)
agents.head()

In [ ]:
frames[['frame_id']] = frames.index
scenes['frame_index_interval'] = scenes['frame_index_interval'].tolist()

In [ ]:
df = pd.DataFrame( columns = ['host','start_time','end_time'])
for i in range(0,100):
    for j in range(scenes['frame_index_interval'][i][0], scenes['frame_index_interval'][i][1]):
        df = df.append(scenes.iloc[i,1:4], ignore_index=True)
df[['frame_id']] = df.index        
        
         

Merging scene dataset and frame dataset using frame_id key

In [ ]:
df = df.merge(frames, on = df.frame_id, how = 'left')
df[['ego_translation_x', 'ego_translation_y', 'ego_translation_z']] = df['ego_translation'].tolist()
df['ego_rotation'] = df['ego_rotation'].tolist()
df.drop(['frame_id_x','frame_id_y','ego_translation'], axis = 'columns', inplace = True)
df.head()

This is the ego motion for the whole scenario.

In [ ]:

#sns.scatterplot(df['ego_translation_x'], df['ego_rotation_y'], df['ego_translation_z'])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df['ego_translation_x'], df['ego_translation_y'], df['ego_translation_z'], c='skyblue', s=60)
ax.view_init(10, 50)
plt.show()


All the agents, encountered by the car during its motion.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(agents['extent_x'], agents['extent_y'], agents['extent_z'], c='green', s=60)
ax.view_init(10, 50)
plt.show()

Agent representation for the first scene.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(agents.iloc[38:86, 5], agents.iloc[38:86, 6], agents.iloc[38:86, 7], c='green', s=60)
ax.view_init(10, 50)
plt.show()

Agent Representation in 2-D graph

In [ ]:
sns.scatterplot(x = agents.centroid_x, y = agents.centroid_y, color = 'blue')

In [ ]:
sns.scatterplot(x = agents[agents.velocity_x != 0].velocity_x, y = agents[agents.velocity_y != 0].velocity_y, color = 'blue')

In [ ]:
#agents.head()
sns.distplot(agents.yaw)

Here the red dot represents the AV, while blue dots depicts the agents with which it is surrounded.

In [ ]:
subsample = agents.iloc[38:86]

#Create combo chart
fig, ax1 = plt.subplots(figsize=(10,6))
 
ax1 = sns.scatterplot(x = subsample.centroid_x, y = subsample.centroid_y, color = 'blue')

#specify we want to share the same x-axis
ax2 = ax1.twinx()
color = 'tab:red'
#line plot creation

ax2 = sns.scatterplot([df.iloc[1,8]], [df.iloc[1,9]], color=color)
ax2.tick_params(axis='y', color=color)
#show plot
plt.show()

In [ ]:
dataset_path = '/kaggle/input/lyft-motion-prediction-autonomous-vehicles/scenes/train.zarr'
zarr_dataset = ChunkedDataset(dataset_path)
zarr_dataset.open()
rast = build_rasterizer(cfg, dm)
train_agent_dataset = AgentDataset(cfg, zarr_dataset, rast)

Using the L5Kit's Rastesizer library.

In [ ]:
data = train_agent_dataset[1]
cfg['raster_params']['map_type'] = "py_satellite"

im = data["image"].transpose(1, 2, 0)
im = train_agent_dataset.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
#draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)

plt.imshow(im[::-1])
plt.show()

There is alot of a work yet to be done wrt to feature engineering and model training.